# Import Libraries and Data

In [ ]:
import ibis
import pandas as pd
import os

In [ ]:
import getpass 
  
try: 
    
    p = getpass.getpass() 
except Exception as error: 
    print('ERROR', error) 
else: 
    print('Password entered:') 

### Import Data from Impala

In [ ]:
hdfs = ibis.hdfs_connect(host=os.environ['HDFS_HOST'], port=50070)

In [ ]:
client_impala = ibis.impala.connect(host=os.environ['IP_IMPALA'], port=21050, \
  hdfs_client=hdfs, user=os.environ['CHANDIMA_LOGIN'], password=getpass.getpass(), \
  auth_mechanism='PLAIN')


In [ ]:
%%time
requete =client_impala.sql('SELECT * FROM open_data.clean_bank')
df = requete.execute()

In [ ]:
df.head()

In [ ]:
df.dtypes

### Check co-relation between the features

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
sns.set(style="whitegrid")

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df.corr())

# Identify the Distribution of Data

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["age"])

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["balance"])

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["estimatedsalary"])

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["tenure"])

In [ ]:
copy = df.copy()
copy.dropna(inplace = True)
sns.distplot(copy["isactivemember"])

Age and Balance are normaly distributed. Tenure and isactivemember are uniformly disributed. These distibutions are useful when to replace null values. Either by mean or median.

# Plots and Visualization

### Customer Exited vs Not Exited

In [ ]:
sns.countplot(x="exited", data=df)
plt.title("Exited vs Not-Exited")
plt.show()

### Churn according to Gender

In [ ]:
sns.countplot('gender',data=df,hue='exited').set_title('Churn with Gender')

### Churn according to Geography

In [ ]:
sns.countplot('geography',data=df,hue='exited').set_title('Churn According to Geography')

### Churn acording to Age Group

In [ ]:
age_cat = df[['age','exited']]

In [ ]:
age_cat.head()

In [ ]:
r = [0, 15, 25, 35, 45, 55,120]
g = ['Children 0 - 15','Teenagers 16-25','Youth 26-35','Adult 36- 45','Mature 46- 55','Old 56+']
age_cat['Age_Category'] = pd.cut(age_cat['age'], bins=r, labels=g)

In [ ]:
df_3 = age_cat.groupby('Age_Category').mean()
df_3.reset_index(inplace=True)
plt.figure(figsize=(6,8))
sns.barplot(x='Age_Category', y='exited', data=df_3,palette='plasma').set_title('Churn vs Age Category')
plt.xticks(plt.xticks()[0],g , rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Feature Engineering

### One-Hot encoding our categorical attributes

In [ ]:
list_cat = ['geography', 'gender']
training_data = pd.get_dummies(df, columns = list_cat, prefix = list_cat)

In [ ]:
training_data.head()

### Feature Rescaling

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()

In [ ]:
#We need to reshape our data since the Scaler takes in arrays
creditscore_train = np.array(training_data["creditscore"]).reshape(-1, 1)
age_train = np.array(training_data["age"]).reshape(-1, 1)
balance_train = np.array(training_data["balance"]).reshape(-1, 1)
estimatedsalary_train = np.array(training_data["estimatedsalary"]).reshape(-1, 1)

training_data["creditscore"] = scaler.fit_transform(creditscore_train)
training_data["age"] = scaler.fit_transform(age_train)
training_data["balance"] = scaler.fit_transform(balance_train)
training_data["estimatedsalary"] = scaler.fit_transform(estimatedsalary_train)

In [ ]:
training_data.head()

# Split the Data Frame

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(training_data, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
X_train = train.drop(['exited','rownumber','customerid','surname'], axis=1).values
y_train = train["exited"].values
X_test = test.drop(['exited','rownumber','customerid','surname'], axis=1).values

In [ ]:
X_train.shape, y_train.shape, X_test.shape

### Import XGBClassifier from xgboost 

We use only one model to prediction

In [ ]:
from xgboost import XGBClassifier

Import libraries to check the accruracy of the model. 
Precision, Recall and F1 score to see how many times you predicted correctly

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### Find out the best hyper parameters 

In [ ]:
from sklearn.model_selection import GridSearchCV

### Create validation Data Set: to avoid overfitting

In [ ]:
X_training, X_valid, y_training, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:


xg_clf = XGBClassifier()

parameters_xg = {"objective" : ["reg:linear"], "n_estimators" : [5, 10, 15, 20]}

grid_xg = GridSearchCV(xg_clf, parameters_xg, scoring=make_scorer(accuracy_score))
grid_xg.fit(X_training, y_training)

xg_clf = grid_xg.best_estimator_

xg_clf.fit(X_training, y_training)
pred_xg = xg_clf.predict(X_valid)
acc_xg = accuracy_score(y_valid, pred_xg)

print("The Score for XGBoost is: " + str(acc_xg))

In [ ]:
precision_xg = precision_score(y_valid,pred_xg, average='weighted')
print("The precision of xgboost is: " + str(precision_xg))
recall_xg = recall_score(y_valid,pred_xg, average='weighted')
print("The recall of xgboost is: " + str(recall_xg))
F1_xg = recall_score(y_valid,pred_xg, average='weighted')
print("The F1 score of xgboost is: " + str(F1_xg))

# Model Performance Comparison

In [ ]:
model_performance = pd.DataFrame({
   
    "Accuracy_Score": [acc_xg],
    "Precision": [precision_xg],
    "Recall" : [recall_xg],
    "F1 Score": [F1_xg],
     "Model": ["XGBoost"]
})


model_performance = model_performance[['Model', 'Accuracy_Score', 'Precision', 'Recall', 'F1 Score']]
model_performance

Fit the model on unseen data

In [ ]:
xg_clf.fit(X_train, y_train)

In [ ]:
result_sumbission = xg_clf.predict(X_test)

In [ ]:
y_test = test["exited"].values

In [ ]:
submission2 = pd.DataFrame({
        "Id": test["customerid"],
        "Customer_name":test["surname"],
        "expected_Exited": y_test,
        "predicted_Exited": result_sumbission.round()
    })

submission2 = submission2[['Id','Customer_name','expected_Exited','predicted_Exited']]

In [ ]:
submission2.head()

### Check the results with confusion_matrix


In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
cm = confusion_matrix(y_valid, pred_xg)
cm

In [ ]:
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels')
ax.set_ylabel('Actual labels'); 
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['0', '1'])
ax.yaxis.set_ticklabels(['1', '0'])

# Save churn_bank_customers.csv file in to HDFS

In [ ]:
from hdfs import InsecureClient
client_hdfs = InsecureClient('http://192.168.56.10:50070', user=os.environ['CHANDIMA_LOGIN'])

In [ ]:
# Writing Dataframe to hdfs
with client_hdfs.write('/user/chandima.pondapelage/Bank/result_csv/churn_bank_customers.csv',encoding = 'utf-8') as writer:
    submission2.to_csv(writer)

# Save churn_bank_customers.csv file in to local file syetem

In [ ]:
submission2.to_csv("path to your local file system/churn_bank_customers.csv",index=False)
